# Identify Intended Journeys Across Domains

Generate `all simple paths` from all source to target nodes. `All simple paths` are all possible paths from source to target nodes, with no repeated nodes.  

It is likely that a large number of simple paths will be returned. Therefore, it is possible to `rank all simple paths`. A variety of methods can be applied, but all come with assumptions. The ranking methods applied are:

- shortest paths 
- pageRank
- closeness centrality
- betweenness centrality 
- eigenvector centrality 

This notebook also allows you to `compare all intended paths to all functional paths`. This provides you with *1)* the structural journeys that appear in the functional journeys, *2)* the structural journeys that do not appear in the functional journeys. 

Note: `pages` and `nodes` are used interchangeably throughout this notebook.

#### Ranking method, assumptions, and conclusions

`Rank via shortest paths`

Method: Orders by shortest path to longest path.

Assumptions: This assumes that content designers design a path from source to target in the shortest, quickest possible way. From talking to Performance Analysts, we know that this is not always the case - sometimes content is designed so that users have to access content over multiple pages, before landing on a target page. 

Conclusions: This seems to work OK for accounts domain. But, there can be many paths with the same length, so would need a way to disentangle these paths. In addition, this is unlikely to scale, due to the above assumption. 

`Rank via pageRank`: 

Method: Calculates pageRank for each page in the network. Sums all pageRanks in a simple path, then creates a proportion in relation to the number of pages in the path. Orders in descending order.

Assumptions: This assumes that important pages are more likely to receive more links from other pages. This may not always be the case, for example, contact us pages are likely to receive many links from all pages, but likely is not part of an intended user journey. 

Conclusions: Given the above assumption, the highly ranked paths include pages that are likely not part of an intended journey (such as 'https://signin.account.gov.uk/contact-us')

`Rank via closeness centrality`:

Method: Average distance all other nodes. Calculates this for each page in the network. Sums all closeness centrality values in a simple path, then creates a proportion in relation to the number of pages in the path. Orders in descending order.

Assumptions: Pages in an intended journey should be close to all other nodes in the network. This is unlikely the case. There may be two unique intended journeys in a network, with no overlapping nodes. 

Conclusions: Similar to pageRank, the hihgly ranked paths include pages that are likely not part of an intended journey, because they link directly to many pages. 

`Rank via betweenness centrality`: 

Method: Calculates shortest paths between all pairs of nodes. Calculates this for each page in the network. Sums all betweenness centrality values in a simple path, then creates a proportion in relation to the number of pages in the path. Orders in descending order.

Assumptions: Therefore, it is often used to find nodes that serve as a bridge from one part of a graph to another. This it is unlikely to suit identifying intended journeys, as intended journeys are likely to be unique, rather than include pages that sit over many intended pages. 

Conclusions: Similar to pageRank, the highly ranked paths include pages that are likely not part of an intended journey, because they link directly to many pages, and therefore are more likely to 'bridge' over many intended journeys. 

`Rank via eigenvector centrality`:

Method: The centrality score for each node is derived from the scores of its incoming neighbours. A high eigenvector score means that a node is connected to many nodes who themselves have high scores. Calculates this for each page in the network. Sums all eigenvector centrality values in a simple path, then creates a proportion in relation to the number of pages in the path. Orders in descending order.

Assumptions: Pages with many hyperlinks to other pages with many hyperlinks will have high scores. This, again, is unlikely for intended journeys as we would expect that intended journey pages will not link to many other pages (for example, a sign-up process). 

Conclusions: Similar to pageRank, the highly ranked paths include pages that are likely not part of an intended journey, because they link directly to many pages that also link to many other pages.

### Import modules

In [ ]:
import os

import compare_journeys
import format_journeys
import networkx as nx
from dotenv import load_dotenv

import src.utils.analyse_intended_journeys
from src.make_data.create_intended_journeys import find_simple_paths, rank_simple_paths

### Load data and assign variables

In [ ]:
load_dotenv()
DIR_DATA_PROCESSED = os.environ.get("DIR_DATA_PROCESSED")
DIR_DATA_INTERIM = os.environ.get("DIR_DATA_INTERIM")
G_structural = nx.read_gpickle(os.path.join(DIR_DATA_PROCESSED, "G_structural.gpickle"))
G_functional = nx.read_gpickle(
    os.path.join(DIR_DATA_INTERIM, "user_journeys_df.gpickle")
)

In [ ]:
# Set all source to target variables that simple paths should be calculated for. Note that a source and target pair
# is considered, for example, source[0] target[0], and source[1] target[1]; rather than source[0] target[1] or
# source[1] target[2]

source = [
    "https://www.gov.uk/email/subscriptions/single-page/new",
    "https://www.gov.uk/email/subscriptions/account/confirm",
]
target = [
    "https://www.gov.uk/email/subscriptions/account/confirm",
    "https://account.gov.uk/delete-account",
]

# Do not include any paths that contain the following pages
exclude_pages = [
    "https://signin.account.gov.uk/contact-us",
    "https://signin.account.gov.uk/signed-out",
    "https://account.gov.uk/delete-account",
]

# Only paths of length <= cutoff are returned. If all paths are to be returned, leave blank
cutoff = 5

# Choose ranking method. See above for method, assumptions, and conclusions for each ranking method
method = "shortest"  # 'shortest', 'pageRank', 'betweenness', 'closeness', 'eigenvector'

### Find and rank all simple paths from source(s) to target(s)

In [ ]:
all_simple_paths = find_simple_paths(
    source, target, G_structural, exclude_pages, cutoff, verbose=1
)

In [ ]:
all_simple_paths_ranked = rank_simple_paths(
    method, G_structural, all_simple_paths, verbose=1
)

### Compare networks

In [ ]:
functional_user_journeys, structural_user_journeys = format_journeys(
    G_functional, G_structural
)

In [ ]:
same_journeys_df, struct_journeys_not_appear = compare_journeys(
    functional_user_journeys, structural_user_journeys
)